In [1]:
#!accelerate launch --mixed_precision fp16 ./train_accel.py --config_name EXP_02

In [2]:
# !CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_HF_V1

In [3]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_HF_V2

In [4]:
# !CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V0

In [5]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V1

In [6]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V2

In [7]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name MATGRAPH

In [8]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name MATGRAPHV2

In [9]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V3

In [10]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V5

In [11]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_HF_V5

In [12]:
#!CUDA_VISIBLE_DEVICES=1 python train.py --config_name BASELINE_HF_V6

In [13]:
#!CUDA_VISIBLE_DEVICES=1 python train.py --config_name BASELINE_HF_V7

In [14]:
#!CUDA_VISIBLE_DEVICES=1 python train.py --config_name BASELINE_HF_V8

In [15]:
#!CUDA_VISIBLE_DEVICES=1 python train.py --config_name BASELINE_HF_V9

In [16]:
#!CUDA_VISIBLE_DEVICES=1 python eval.py --config_name BASELINE_HF_V8FTEVAL

In [17]:
def set_gpu_environ():
    """Sets CUDA_VISIBLE_DEVICES to those under minimal memory load.
    Meant to be used in notebooks only.
    """
    import os
    import subprocess
    query = subprocess.check_output(['nvidia-smi', '--query-gpu=memory.used', '--format=csv']).decode().split('\n')[1:-1]
    utilization = [int(x.replace(" MiB", "")) for x in query]
    free = [i for i in range(len(utilization)) if utilization[i] == min(utilization)]
    set_visible = ",".join([str(i) for i in free])
    os.environ["CUDA_VISIBLE_DEVICES"] = set_visible
set_gpu_environ()

In [18]:
import os
import random
import numpy as np
import torch
import config

def seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    

def train(cfg):
    seed()
    custom_model = cfg.MODEL_NAME()
    if cfg.MODEL_WTS:
        print(f"Loading model weights from {cfg.MODEL_WTS}")
        custom_model.load_state_dict(torch.load(cfg.MODEL_WTS))
    opt = cfg.OPT(
        custom_model.parameters(), lr=cfg.LR, weight_decay=cfg.WD
    )
    loss_func = cfg.LOSS_FUNC()
    scheduler = cfg.SCHEDULER(
        opt,
        num_warmup_steps=cfg.WARM_UP_PCT * cfg.EPOCHS,
        num_training_steps=cfg.EPOCHS,
    )



    cfg.FIT_FUNC(
        epochs=cfg.EPOCHS,
        model=custom_model,
        loss_fn=loss_func,
        opt=opt,
        metric=cfg.METRIC,
        config = cfg,
        folder=cfg.FOLDER/cfg.EXP_NAME,
        exp_name=f"{cfg.EXP_NAME}",
        device=cfg.DEVICE,
        sched=scheduler,
    )
    
def main(config_name):
    configs = eval(f"config.{config_name}")
    print(f"Training with config: {configs.__dict__}")
    os.makedirs(configs.FOLDER/configs.EXP_NAME)
    train(configs)

/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


graphnet: INFO     2023-02-17 05:19:43 - get_logger - Writing log to logs/graphnet_20230217-051943.log


In [ ]:
main('BASELINE_HF_V15_FT')

Training with config: {'__module__': 'config', 'EXP_NAME': 'EXP_29_FT', 'MODEL_WTS': '/opt/slh/icecube/RESULTS/EXP_29/EXP_29_5.pth', 'LR': 0.0001, 'LOSS_FUNC': <class 'icecube.models.VonMisesFisher3DLossEcludeLossCosine'>, 'FIT_FUNC': <function fit_shufllef32 at 0x7f8983eb78c0>, 'EPOCHS': 6, 'TRN_DATASET': <class 'icecube.dataset.HuggingFaceDatasetV12'>, 'VAL_DATASET': <class 'icecube.dataset.HuggingFaceDatasetV12'>, 'COLLAT_FN': <function collate_fn_v1 at 0x7f8983eb7e60>, 'MODEL_NAME': <class 'icecube.models.IceCubeModelEncoderSensorEmbeddinngV4'>, 'BATCH_SIZE': 768, '__doc__': None}
Loading model weights from /opt/slh/icecube/RESULTS/EXP_29/EXP_29_5.pth


/opt/conda/lib/python3.7/site-packages/datasets/arrow_dataset.py:1536: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  FutureWarning,
wandb: Currently logged in as: drhb (kaggle-hi). Use `wandb login --relogin` to force relogin


epoch,train_loss,valid_loss,val_metric


trn_range: [300, 400]


/opt/conda/lib/python3.7/site-packages/datasets/arrow_dataset.py:1536: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  FutureWarning,
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Better model found at epoch 0 with value: 1.0040115118026733.
   epoch  train_loss  valid_loss     metric
0      0    0.965695    0.962668  1.0040115
trn_range: [400, 500]


/opt/conda/lib/python3.7/site-packages/datasets/arrow_dataset.py:1536: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  FutureWarning,
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
torch.cuda.current_device()

In [ ]:
k

In [ ]:
k